In [13]:
import re
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [5]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).collect()

In [9]:
def hledej(text, n=3, delka=200, frejm=df, sestupne=False):
    nalezy = frejm.sort(by="datum", descending=sestupne).filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [17]:
hledej("(?i)antikonc")

Kdo: Ministr veř. zdravotnictví a tělesné výchovy Šrámek
Kde: 1920ns_ps_stenprot_364schuz_s364002.htm
Kdy: 1925-09-24
Co: gulovati počet porodních pomocnic v určitém okrsku - rayonování - po případě i ustanovovati porodní pomocnice u obcí s určitými fixními platy. Konečně často upozorňují úřady na možnost zákazu prodeje antikoncepčních prostředků a na možnost censury pseudozdravotnické literatury. (Různé výkřiky.) Předseda (zvoní): Žádám o klid.
***
Kdo: Posl. Procházková
Kde: 1964ns_stenprot_018schuz_s018024.htm
Kdy: 1968-01-11
Co: ch školách, v dětských domovech, zanedbávání péče o děti, problémy opuštěných děti a nesnáze s osvojováním, nebo sociálně ekonomické problémy naší mladé generace, populační otázky a s tím související antikoncepce, umělé přerušení těhotenství, pohlavní nemoci a dokonce i protispolečenská činnost, jako je prostituce, a dospěli jsme i ke konkrétním návrhům. Dovolujeme si proto nabídnout své poznatky a zkušen
***
Kdo: Posl. Poledňák
Kde: 1964ns_stenprot_023schu

In [19]:
hledej("(?i)internet")

Kdo: Poslankyně Květoslava Čelišová
Kde: 1993ps_stenprot_037schuz_s037019.htm
Kdy: 1995-12-07
Co: čnou překážkou zůstane stavební povolení. Opravdu chceme, aby o tak strategicky důležité věci, jakou zcela jistě kabelové rozvody jsou, a nejde pouze o televizní vysílání, ale i o informační dálnici, Internet apod., rozhodoval jeden úředník stavebního odboru obecního úřadu? Jinou věcí je, že v kabelových rozvodech mohou vysílat, resp. vytvářet programy i ti, kteří sami vlastníky rozvodu nejsou. O liberali
***
Kdo: Ministr kultury ČR Jaromír Talíř
Kde: 1996ps_stenprot_010schuz_s010026.htm
Kdy: 1997-03-27
Co: najdeme řešení pro jednotlivá zde definovaná témata a pro definované problémové okruhy. Myslím si, že ještě bude nutné, a zapomněl jsem tu uvést, že tato legislativa se také dotkne datových sítí typu Internet a že tady bude třeba také definovat určité terminologie a jejich řešení. Kromě toho tento odbor se také musí zabývat záležitostí nejenom elektronických médií, ale např. tiskovým zá

In [27]:
hledej(r"(?i)uměl[áéo]\w{0,5} intelig")

Kdo: Předseda vlády ČR Andrej Babiš
Kde: 2017ps_stenprot_023schuz_s023006.htm
Kdy: 2018-11-23
Co: ci, kdy si skutečně můžeme vybírat. Je jasná naše ekonomická strategie. Věda a výzkum a finální výroba. V Bruselu u předsedy Evropské komise jsem lobboval za to, abychom získali centrum excelence pro umělou inteligenci. To je pro nás strašně důležité. Novelu zákona o nemocenském pojištění zavedeme od července příštího roku. Zavedeme elektronickou neschopenku, další důležitý krok k elektronizaci zdravotnictví. T
***
Kdo: Předseda vlády ČR Andrej Babiš
Kde: 2017ps_stenprot_026schuz_s026245.htm
Kdy: 2019-01-31
Co: kotvení digitalizace; v prosinci 2018 představení Sonia, bankovní identita pro státní správu, po realizaci získáme kmen 4 milionů klientů do státní správy; v prosinci 2018 schválena strategie využití umělé inteligence v rámci našeho programu Česká republika, země pro budoucnost; dále v prosinci 2018 memorandum k využití technologie blockchain; v prosinci 2018 digitálně technická map

In [29]:
hledej(r"(?i)bitcoin")

Kdo: Poslanec Zbyněk Stanjura
Kde: 2013ps_stenprot_033schuz_s033113.htm
Kdy: 2015-10-07
Co: sely. A je to nevhodný nástroj a my se snažíme vysvětlit, proč je nevhodný. Já si nechám, řekl bych, možná ty lehčí otázky, na to třetí čtení. Například by mě zajímalo, v kolika restauracích se platí bitcoiny, když myslíme na 
***
Kdo: Poslanec Adolf Beznoska
Kde: 2013ps_stenprot_047schuz_s047158.htm
Kdy: 2016-05-31
Co: v i na nás, na běžné občany. Částka 10 tisíc eur se může zdát relativně vysoká, ale předesílám, že některé evropské země tuto částku již mají poloviční. Návrh začal řešit i virtuální měny, jako třeba bitcoiny, a zahrnuje osoby poskytující služby spojené s virtuální měnou do okruhu povinných osob, kterým vyplývají povinnosti vyplývající z tohoto zákona. To samozřejmě může snížit jistou atraktivitu tohoto 
***
Kdo: Předseda vlády ČR Andrej Babiš
Kde: 2017ps_stenprot_019schuz_s019103.htm
Kdy: 2018-09-13
Co: Dobrý den. Já jsem čekal, že piráti nám něco ukážou. Vy jste měli tolik řečí

In [51]:
hledej(r"(?i)www")

Kdo: Místopředseda vlády Pavel Mertlík
Kde: 1998ps_stenprot_013schuz_s013242.htm
Kdy: 1999-05-20
Co:  tím, co obsahuje hospodářská část zprávy o stavu společnosti. Myslím, že nemá cenu vypočítávat to, co je ve zprávě řečeno. Zpráva byla včas publikována jak na internetu, na vládním serveru na adrese www.vlada.cz, tak v Hospodářských novinách, což je všeobecně čtený a respektovaný deník. Sněmovna ji samozřejmě dostala včas. Chtěl bych říci, že dnes prožíváme hospodářskou recesi, která po roce 1990 ne
***
Kdo: Ministr bez portfeje ČR Karel Březina
Kde: 1998ps_stenprot_036schuz_s036252.htm
Kdy: 2001-05-24
Co: m na starosti oblasti informačních a komunikačních technologií, tak všechny dokumenty, které vláda v této oblasti přijímá, jsou přístupné na internetu, a to jak na internetových stránkách Úřadu vlády www.vlada.cz, anebo na internetových stránkách Úřadu pro veřejné informační systémy 
***
Kdo: Předseda vlády ČR Miloš Zeman
Kde: 1998ps_stenprot_037schuz_s037117.htm
Kdy: 2001-06-28
Co: 

In [53]:
hledej(r"(?i)faceboo")

Kdo: Poslanec Ondřej Liška
Kde: 2006ps_stenprot_059schuz_s059199.htm
Kdy: 2009-06-17
Co: otvrzeno či vyvráceno, zda maturita je, nebo není dobře připravena. To znamená aby byl důsledně hájen zájem těch, kteří budou první generací, která bude skládat státní maturitní zkoušku. To, že se na facebooku přihlásilo několik desítek tisíc mladých lidí, kteří mají strach z něčeho nového, o čem přesně všechno nemohou vědět, protože je to velmi složitý proces, do kterého vidí skutečně jen několik desíte
***
Kdo: Poslankyně Olga Zubová
Kde: 2006ps_stenprot_060schuz_s060071.htm
Kdy: 2009-09-09
Co: louchat jim. Státní maturitu odmítá stále víc lidí. Svůj nesouhlas vyjádřilo přes 20 tisíc lidí v petici za odklad státních maturit. Bezmála 60 tisíc členů obsahuje každá ze dvou skupin sociální sítě Facebook, které vyjadřují svůj nesouhlas s projektem, který jste vy, poslanci, nebyli schopni obhájit.
***
Kdo: Poslanec Zdeněk Škromach
Kde: 2006ps_stenprot_064schuz_s064068.htm
Kdy: 2009-10-21
Co: utečně zač

In [55]:
hledej(r"(?i)whatsapp")

Kdo: Poslanec Marian Jurečka
Kde: 2017ps_stenprot_026schuz_s026307.htm
Kdy: 2019-02-12
Co: ci předtím s migranty ani s žádným ilegálním, řekněme, lídrem nějaké skupiny, který by to organizoval, v kontaktu nebyl. Byl v kontaktu pouze s rodinou. Dokonce se prověřovala třeba i komunikace přes WhatsApp. Také tam není to, co by tam mělo být, a to znamená, že by mu za tu službu bylo zaplaceno buď předem, nebo poté. Takže to jsou silné argumenty pro to, abychom se mohli domnívat, že to řízení nebylo o
***
Kdo: Poslanec Ondřej Profant
Kde: 2017ps_stenprot_062schuz_s062340.htm
Kdy: 2020-11-12
Co: ernet. Proto máme v Ústavě právo na soukromí. Proto mě velmi rmoutí, že na výboru pro evropské záležitosti 4. listopadu byla vzata na vědomí výjimka z ePrivacy směrnice, aby poskytovatelé služeb typu WhatsApp a podobně prolamovali šifrování. To je zcela anachronické, jde to proti současné době. Chci se zeptat, jestli s tímto souhlasíte. Jestli si myslíte, že má být plošně narušováno soukromí uživatelů 

In [57]:
hledej(r"(?i)tiktok")

Kdo: Ministr školství, mládeže a tělovýchovy ČR Vladimír Balaš
Kde: 2021ps_stenprot_035schuz_s035119.htm
Kdy: 2022-09-08
Co: tak se snažíme navázat kontakt se všemi potenciálními ukrajinskými asistenty nebo pedagogy. Problém byl v tom, že oni používají jiné sociální sítě, než na jaké my jsme zvyklí, například Telegram nebo TikTok. Ale máme i přehled o tom, v jakých krajích a v jakých ORP ten problém může být, je to spíše v těch větších aglomeracích, jak jste zmínil. A máme samozřejmě tendenci také s nimi spolupracovat a zvyšo
***
Kdo: Ministr školství, mládeže a tělovýchovy ČR Vladimír Balaš
Kde: 2021ps_stenprot_035schuz_s035124.htm
Kdy: 2022-09-08
Co:  situace je velmi pestrá. My se snažíme zjistit další informace. Já jsem říkal, že jeden problém je v tom, že fungují Ukrajinci na jiných sociálních sítích, než my jsme zvyklí, je to hlavně Telegram, TikTok, takže jsme zaměřili ty kontakty i na tyhlety sítě, a nejenom že zjišťujeme, že k některým rodičům se nedostaly informace, jak mají,

In [59]:
hledej(r"(?i)fake news")

Kdo: Ministr zdravotnictví ČR Miloslav Ludvík
Kde: 2013ps_stenprot_058schuz_s058020.htm
Kdy: 2017-05-10
Co: e překvapovat, jakým způsobem se v téhle zemi vytvářejí nebo vyrábějí kauzy. Z tohoto se udělá kauza. Finále je co? Nic. Kdybych byl blonďák a seděl jsem o něco západněji, tak bych to nazval typickou fake news. Ale já touto cestou cítím, a omlouvám se, že tady zdržuji, cítím trochu povinnost se za prvé obhájit a za druhé říci, že tenhle způsob obchodování s informacemi je strašný, a skoro je mi líto, že to
***
Kdo: Poslankyně Helena Langšádlová
Kde: 2013ps_stenprot_057schuz_s057371.htm
Kdy: 2017-06-08
Co:  Koudelka na serveru HlídacíPes. Z rozhovoru vyplývají mnohé negativní skutečnosti. Jedná se o nárůst webových stránek, na kterých se velmi často setkáváme s proruskou propagandou, ale i s tím, že se fake news vyskytují i ve zdánlivě seriózních médiích. Považuji za velmi nebezpečné, že často dochází ke zpochybňování základních demokratických principů, zpochybňování nejen domácíc

In [61]:
hledej(r"(?i)hoax")

Kdo: Poslanec Jan Klán
Kde: 2013ps_stenprot_055schuz_s055275.htm
Kdy: 2017-03-03
Co:  Aldoux Huxley a jeho kniha Konec civilizace, která říká, že budeme žít v takové změti různých informací a budeme v tom mít takový zmatek, že nebudeme vědět, co je pravda a co ne. A to souvisí s těmi hoaxy a faky, že je takové množství dat, že nevíme, které je pravdivé. Takže elektronické občanské průkazy mají spojitost s kontrolovanou společností, protože někdo někde bude o mně mít další dodatečnou k
***
Kdo: Místopředseda vlády ČR Pavel Bělobrádek
Kde: 2013ps_stenprot_060schuz_s060003.htm
Kdy: 2017-09-05
Co: e, anebo skutečně používá fake news, jak naši europoslanci hlasovali ohledně uvalení sankcí na Českou republiku za nepřijímání migrantů. Prostřednictvím řetězových e-mailů se mezi českými občany šíří hoax o tom, jak nám čeští vlastizrádci z Evropského parlamentu hlasovali pro uvalení sankcí na Českou republiku za nepřijímání migrantů. Tato skutečnost vznikla po blogu europoslance Petra Macha za S

In [63]:
hledej(r"(?i)dezinf")

Kdo: Prezident Gottwald pri tejto príležitosti povedal
Kde: 1948ns_stenprot_040schuz_s040003.htm
Kdy: 1950-02-22
Co: italistických štátoch, si nijako vojnu nepraje. A to je príčina, prečo podnecovatelia novej vojny vrhajú sa naň s obrovským aparátom. "V Spojených štátoch amerických vládne prekvapujúci stav verejnej dezinformovanosti", povedal raz prezident Roosevelt. A s týmto stavom zámerne udržovanej 
***
Kdo: Předsedkyně Sněmovny lidu MUDr. S. Pennigerová
Kde: 1969fs_sl_stenprot_004schuz_s004002.htm
Kdy: 1969-10-15
Co: í mnozí pocit, že byli oklamáni. Avšak nevidí ještě dosud skutečné viníky, a to ty, kteří s nimi kalkulovali a v období vypjatých politických situací je měli připravené jako želízka v ohni. Politická dezinformace je u mládeže zvlášť silná. Je u ní i silná deziluze. Chybí jí dnes vzory, ideály, někdo nebo něco, za čím by mohla jít a čemu by mohla věřit. Není pro mladého člověka jistě jednoduché, jestliže
***
Kdo: Poslanec S. Bartaloš
Kde: 1971fs_slsn_stenprot_004schuz_

In [75]:
hledej(r"(?i)[^\w]hovn[aoye]\w{0,1}[^\w]")

Kdo: Poslanec Jan Volný
Kde: 2021ps_stenprot_027schuz_s027055.htm
Kdy: 2022-06-23
Co: hodil se nám sem zpovídat ten ředitel. To byl poctivý a neskutečně pracovitý kluk, holt možná někde víc agilní, než bylo zdrávo. To je pravda. Ale tady to vytahovat a říkat, vy jste to dělali úplně na hovno, my to budeme dělat líp? No, je to politika. Děkuji za slovo.
***
Kdo: Poslanec Roman Kubíček
Kde: 2021ps_stenprot_115schuz_s115127.htm
Kdy: 2024-10-23
Co: , tak to prostě je. Máte jezdit do toho Bruselu a máte říkat, že to nemá být klasický finanční derivát. Vy máte říkat, že si to mají vybírat ti eminenti. Když s tím nic neděláte, tak jste tam úplně na hovno! Pardon.
***


In [77]:
hledej(r"(?i)Petr Pavel")

Kdo: Poslanec Petr Hulinský
Kde: 2010ps_stenprot_040schuz_s040166.htm
Kdy: 2012-06-12
Co: k stojí před úkolem adaptovat se co nejdříve na nové potřeby a cíle bezpečnostní a branné politiky NATO a Evropské unie. Věřím, že k tomu může přispět i nový náčelník Generálního štábu generálporučík Petr Pavel. Jedno je ale v případě Afghánistánu pro armádu velmi důležité. Naši vojáci obstáli v dlouholeté afghánské misi se ctí, a proto je třeba našim vojákům za vzornou reprezentaci České republiky poděkova
***
Kdo: Poslankyně Jana Černochová
Kde: 2013ps_stenprot_057schuz_s057039.htm
Kdy: 2017-05-17
Co: icie, ze kterého během dvou týdnů z podplukovníka udělal generála. Mohl si vybrat svého ředitele Vojenského zpravodajství. Mohl si vybrat svého náčelníka Generálního štábu, protože končil pan generál Petr Pavel a novým náčelníkem Generálního štábu se stal pan generál Bečvář. Můžete mi říct, který z ministrů - a nemluvě o těch dalších věcech, o kterých tady hovořil kolega Ženíšek, že je ekonomický r

In [79]:
hledej(r"(?i)koronavir")

Kdo: Poslanec Bohuslav Svoboda
Kde: 2017ps_stenprot_040schuz_s040160.htm
Kdy: 2020-01-28
Co: Já to zkusím znovu. Navrhuji zařazení bodu na pořad dnešního jednání jako první bod Informace vlády ČR k opatřením souvisejícím se šířením koronaviru. Už jsem několikrát řekl, není mým cílem vyvolávat paniku, to v žádném případě, ale Sněmovna i lidé v České republice by měli dostat ujištění, že je vláda schopna reagovat, pokud se věci nebudou vyv
***
Kdo: Předseda PSP Radek Vondráček
Kde: 2017ps_stenprot_040schuz_s040162.htm
Kdy: 2020-01-28
Co: návrh nebyl přijat. Pak je zde návrh pana poslance Bohuslava Svobody, abychom do pořadu schůze zařadili nový bod, kterým je Informace vlády, případně ministra zdravotnictví, o opatřeních proti šíření koronaviru. Zahajuji hlasování. Ptám se, kdo je pro zařazení nového bodu. Kdo je proti? V hlasování číslo 115 přihlášeno 165, pro 72, proti 3. Tento návrh nebyl přijat. O pevném zařazení tedy už nebudeme hlaso
***
Kdo: Předseda vlády ČR Andrej Babiš
Kde: 201

In [87]:
hledej(r"DVD")

Kdo: Poslanec Zdeněk Švrček
Kde: 1998ps_stenprot_017schuz_s017060.htm
Kdy: 1999-10-13
Co: žnost upravit si vše tak, aby z ložné plochy jakéhokoliv auta mohl kdokoli prodávat cokoli, avšak rád bych svým dalším návrhem ponechal právo pojízdným prodejnám firem, jako jsou Ledík, Family Frost, DVD nebo MAN, prodávat dosavadním způsobem a nevystavoval je možnému nátlaku použitím dosud navržené úpravy.
***
Kdo: Poslanec Aleš Rozehnal
Kde: 1998ps_stenprot_022schuz_s022018.htm
Kdy: 2000-02-22
Co: ýlučného distribučního práva je odůvodnitelné v rámci jednotného trhu, nikoliv však ve vztahu k třetím státům. Na tomto principu je založen i celosvětový systém distribuce a ochrany digitálních disků DVD, jejichž volná distribuce je technickými opatřeními omezena pouze na okruh stanoveného regionu. Teritoriální princip se uplatňuje též v Evropské unii, kde je zachováno absolutní právo národního distr
***
Kdo: Poslankyně Kateřina Dostálová
Kde: 2002ps_stenprot_042schuz_s042032.htm
Kdy: 2005-03-23
Co: m 

In [89]:
hledej(r"(?i)delulu")

In [101]:
hledej(r"(?i)[^\w]Gaz[aey][^\w]")

Kdo: Poslanec SL J. Lichner
Kde: 1976fs_slsn_stenprot_013schuz_s013012.htm
Kdy: 1979-10-24
Co: ská zmluva uvoľnila izraelskej vláde ruky. Dokazuje to jednak izraelská politika na okupovaných územiach, postoj na rokovaniach o autonómii pre arabské obyvateľstvo na západnom brehu Jordánu a v pásme Gazy. Zvlášť provokatívna a nebezpečná je izraelská agresia proti susednému Libanonu, ktorej súčasťou je pokus o genocídu palestínskeho ľudu, toľko skúšaného v posledných desaťročiach. Z doterajšieho prieb
***
Kdo: Poslanec Josef Krejsa
Kde: 1996ps_stenprot_017schuz_s017041.htm
Kdy: 1997-12-10
Co: dloužen do 31. ledna 1998. Konec citátu. Tedy výzva bez sankcí, vše zůstalo nevyslyšeno. Další rezoluce Rady bezpečnosti je nejčerstvějšího data a týká se zákazu výstavby bytů na okupovaných oblastech Gazy. Tuto rezoluci vydala Rada bezpečnosti opět na rozdíl od rezolucí zaslaných Iráku a Libyi bez vyhlášení sankcí proti Izraeli. Proto se ptám, kde je ta spravedlnost, kde je ta demokracie. Jak to, že a

In [103]:
hledej(r"(?i)irozhlas")

Kdo: Předseda vlády ČR Andrej Babiš
Kde: 2017ps_stenprot_007schuz_s007101.htm
Kdy: 2018-03-01
Co:  Ne, tady je: Murína kvůli kauze úniku informací Bereta vyslýchali policisté. Důstojník uvedl, že se stýkal s Gáboríkem, jehož policie stíhá kvůli obchodování s utajovanými informacemi. Četl jste to? iRozhlas! Antibabišovský server! Každý den kopou do Babiše. (Hovoří pobaveně.) No tak to je strašný, ne, co řekli. Tak si to nastudujte, pane kolego, a neříkejte, že je to Čapí hnízdo. A nikoho nebudeme dopor
***
Kdo: Poslanec Dominik Feri
Kde: 2017ps_stenprot_016schuz_s016098.htm
Kdy: 2018-06-28
Co: alá není, tak nemůže osvětlit i záležitosti kolem výkonu koncipientury 300 km od svého bydliště, nebo třeba osvětlit a uvést na pravou míru informace, které vyšly před půl hodinou v článku na serveru iROZHLAS s titulkem Nastupující šéfka justice Taťána Malá podle expertů opisovala v diplomové práci. Věřím, že paní ministryně bude na interpelace v budoucnu chodit častěji a nebude navštěvovat pouze 

In [35]:
hledej(r"(?i)masturb")

Kdo: Poslanec Karel Vymětal
Kde: 1998ps_stenprot_046schuz_s046420.htm
Kdy: 2002-02-14
Co: Děkuji za slovo. Ano, chci předložit procedurální návrh. Dámy a pánové, nechci mluvit o mikrofonové masturbaci, i když by to někdo mohl tak cítit, já to tak necítím. Já vnímám, že do současné chvíle byla přednesena řada velmi rozsáhlých a hlubokých projevů, jako například pana poslance Sloty, promiňte Neča
***
Kdo: Poslanec Petr Kudela
Kde: 2013ps_stenprot_027schuz_s027122.htm
Kdy: 2015-04-30
Co: přes užívání sladkostí k čištění zubů, celkovou hygienu, škodlivost kouření a nebezpečí drog až - cituji - jací se mi líbí kluci, jaké se mi líbí holky, menstruace, měsíčky, menzes, čmýra, onanování, masturbace, prstění. Prosím, dětem v 5. třídě! Informovat školáky o stavbě lidského těla a o jeho fyziologii a reprodukci je zcela namístě, ale je potřeba to dělat kultivovanou formou. Forma, která je v diář
***
Kdo: Poslanec Andrej Babiš
Kde: 2021ps_stenprot_124schuz_s124074.htm
Kdy: 2024-12-11
Co: íklad mla

In [39]:
hledej(r"(?i)Donald\w{0,3} Trump")

Kdo: Místopředseda vlády a ministr vnitra ČR Radek John
Kde: 2010ps_stenprot_013schuz_s013064.htm
Kdy: 2011-02-03
Co:  pan Jahodář měl ovlivnit nějaký tendr. Podle mých informací se žádný tendr na přepravu hotovosti a cenin v České poště nepřipravuje. Takže to je úplně stejné, jako kdybyste říkal, že pan Jahodář pro Donalda Trumpa chce ovlivnit nějakou zakázku, jenomže ona není. Tak co budeme dělat, pane doktore? Všechny ty vaše řeči o mém soukromém podnikání - a ještě mě přistrkujete do MEDEA KULTUR, kde jsem nikdy nebyl, ne
***
Kdo: Místopředseda vlády ČR a ministr financí Andrej Babiš
Kde: 2013ps_stenprot_043schuz_s043004.htm
Kdy: 2016-03-23
Co: , aby realizovali smysl svého života a předali své dílo svým dětem, je zamezeno se podílet na veřejném životě ve prospěch všech občanů. Přitom všude na světě je trend opačný. V Americe máme ve finále Donalda Trumpa. V Americe byl nejúspěšnější starosta New Yorku podnikatel Michael Bloomberg, kterého jsem potkal minulý rok ve Francii, a ten m

In [41]:
hledej(r"(?i)proletář",sestupne=True)

Kdo: Poslankyně Alena Schillerová
Kde: 2021ps_stenprot_115schuz_s115025.htm
Kdy: 2024-10-22
Co: chterová - teď tady není, takže nemusím říkat prostřednictvím - ještě pořád je místopředsedkyní Sněmovny? S ministerským platem, s autem s řidičem, s aparátem? Pořád tam - jak je to možné? Vy, takoví proletáři! (Pobavení v sále.) Tak já očekávám rezignaci v co nejkratší době! Děkuju. (Poslanci volají, potlesk z lavic hnutí ANO.)
***
Kdo: Poslankyně Eva Dundáčková
Kde: 2002ps_stenprot_054schuz_s054139.htm
Kdy: 2006-03-10
Co: nost v lidově demokratickém Československu - Rudé právo 1956, autor Zdeněk Jičínský: Buržoazní právo a s ním i licoměrná zvetšelá buržoazní ideologie o nadtřídním právu je odklizena na smetiště dějin proletářskou revolucí, v níž se vytváří nové, revoluční 
***
Kdo: Poslanec Josef Vondruška
Kde: 2002ps_stenprot_051schuz_s051607.htm
Kdy: 2005-12-20
Co: o jednání v úterý, kdy opravdu jsem zaregistroval, že asi jenom dvě expresivní slova ze slovníku dlaždiče tu chyběla, tak d

In [43]:
hledej(r"(?i)skinhead",sestupne=True)

Kdo: Místopředseda PSP Ivan Langer
Kde: 2002ps_stenprot_047schuz_s047011.htm
Kdy: 2005-09-20
Co: tázku, v čem se lišil právní stav věci, když se tak zásadně lišil faktický stav policejního zákroku. Otázka osmá: Jaký je důvod pro odlišný přístup policie v případech shromáždění neonacistů, průvodu skinheadů a anarchistů ulicemi, při shromáždění komunistů na Letné, při akci CzechTek, při demonstraci u budovy Ministerstva vnitra a posprejování její fasády?
***
Kdo: Místopředseda PSP Ivan Langer
Kde: 2002ps_stenprot_046schuz_s046007.htm
Kdy: 2005-08-16
Co:  nás všechny musí zajímat odpověď na otázku, zda policie měří v této zemi všem podle jednoho metru, tedy proč v případě CzechTeku postupovala takto, proč postupuje odlišným způsobem v případě průvodu skinheadů a anarchistů ulicemi našich měst, proč zareagovala na shromáždění Komunistické strany Čech a Moravy takovým způsobem, jakým zareagovala, proč zareagovala, resp. nezareagovala v případě demonstrace u
***
Kdo: Místopředseda vlády a mi

In [45]:
hledej(r"Paroub",sestupne=True)

Kdo: Poslanec Andrej Babiš
Kde: 2021ps_stenprot_146schuz_s146011.htm
Kdy: 2025-06-25
Co: nou, zákulisní osobnost brněnské ODS i potenciální hrozbu. Topolánek i Nečas měli z Blažka obavy. Kdo by neměl, takový kmotr! V letech 2008 až 2013 dominovali ODS a celostátní politice tři muži: Jiří Paroubek, Petr Nečas a Mirek Topolánek. Oba zmínění premiéři za ODS Blažkovi nikdy nevěřili, museli ale respektovat jeho vliv a schopnost budovat nemožné koalice ve složitém prostředí, často protichůdných z
***
Kdo: Ministr financí ČR Zbyněk Stanjura
Kde: 2021ps_stenprot_136schuz_s136063.htm
Kdy: 2025-04-16
Co: a neprosadila opozice. To v době, mám to od pamětníků stojedničky, mám pocit, že vláda, to bylo to období, kdy měli sociální demokraté tři premiéry, to znamená Vladimír Špidla, Stanislav Gross a Jiří Paroubek, jich bylo přesně 101, tak aby se shromáždili v konkrétní čas na hlasování, tak si vymysleli nástroj a prosadili do zákona hodiny pro třetí čtení, 9 až 14 středa, pátek. Byl to tehdy nástro

In [105]:
hledej(r"(?i)parní\w{0,5} stroj",sestupne=True)

Kdo: Poslanec Václav Klaus
Kde: 2017ps_stenprot_051schuz_s051024.htm
Kdy: 2020-06-18
Co: tě, všechno možné se vezlo, ale že by tatínek jezdil do práce automobilem, bylo vyloučené, protože ten benzin byl daleko dražší. Já ještě řeknu jednu věc. Když jsme u těch technologií, kupodivu třeba parní stroje byly docela efektivní. Když jsem mluvil o té chalupě, tak do Benešova jezdila elektrická lokomotiva, ale pak už elektrika nebyla, nevím, jestli je dneska teda, asi jo, a pak se jezdilo parní lokomot
***
Kdo: Ministryně pro místní rozvoj ČR Klára Dostálová
Kde: 2017ps_stenprot_016schuz_s016092.htm
Kdy: 2018-06-28
Co: kraticky dané nebo zbytečně administrativně nákladné nebo jsou už přežité. Už jsem to i jednou říkala na výboru pro veřejnou správu, že my se například zabýváme zákonem z roku "devatenáct set něco" o parních strojích. Samozřejmě dneska 
***
Kdo: Poslanec Jan Zahradník
Kde: 2017ps_stenprot_012schuz_s012245.htm
Kdy: 2018-04-19
Co:  účinnosti o 30 %, nebo zda je potřeba být ambici

In [107]:
hledej(r"(?i)telegra[mf]",sestupne=True)

Kdo: Poslanec Vladimír Zlínský
Kde: 2021ps_stenprot_144schuz_s144098.htm
Kdy: 2025-06-18
Co: rmací, byl vynález knihtisku, který pak vedl k tomu, že se výrazně zvýšila vzdělanost populace, schopnost číst a psát, a to samozřejmě podnítilo vědecký a technický rozvoj, který dále vedl k vynálezu telegrafu, rádiového a televizního vysílání, které umožňovaly rychlé, většinou tedy jednosměrné šíření typu informací směrem k jejich uživatelům a vedly k posílení jak demokratických, tak propagandistických p
***
Kdo: Poslanec Vladimír Zlínský
Kde: 2021ps_stenprot_139schuz_s139025.htm
Kdy: 2025-05-27
Co: , že se nestala, ale stala a bylo to v roce 1859. Jmenuje se to takzvaná Carringtonova událost. A tehdy měl svět velké štěstí, protože jediné elektrické přístroje, které se používaly, v podstatě byly telegrafy, takže tehdy zcela vyhořely rozvody 
***
Kdo: Poslanec Patrik Nacher
Kde: 2021ps_stenprot_129schuz_s129046.htm
Kdy: 2025-02-14
Co: Děkuji, pane místopředsedo. Já nevím, kde začít všude. Já to

In [109]:
hledej(r"(?i)pager",sestupne=True)

In [111]:
hledej(r"(?i)magnetof",sestupne=True)

Kdo: Poslanec Patrik Nacher
Kde: 2021ps_stenprot_113schuz_s113095.htm
Kdy: 2024-10-03
Co: ešli jsme od telefonu s vytáčenými čísly, s operátorem spojujícím dálkové hovory a placených telefonních budek k videohovorům s přáteli kdekoliv po světě. Postupně jsme přešli z vinylových desek přes magnetofonové pásky a CD k on-line hudbě a také od ručně psaných dopisů a telegramů k e-mailům a WhatsAppu. Jsme lidé analogového dětství a digitální dospělosti, od poslechu živých her v třílampovém rádiu a s
***
Kdo: Poslanec Tomio Okamura
Kde: 2021ps_stenprot_065schuz_s065119.htm
Kdy: 2023-05-18
Co: Vážený pane premiére, podívejme se, co jste jménem ODS a koalice SPOLU říkal lidem ještě před pár týdny v televizi NOVA. (Pouští magnetofonový záznam z rozhovoru, kdy na řadu otázek, zda se budou zvyšovat daně, pan premiér odpovídá: Nebudeme zvyšovat daně. Tečka.) A teď se podívejme, co jste slibovali lidem před volbami. (Ukazuje různé
***
Kdo: Poslanec Zbyněk Stanjura
Kde: 2013ps_stenprot_012schuz_s0121

In [113]:
hledej(r"(?i)disket",sestupne=True)

Kdo: Poslanec Andrej Babiš
Kde: 2021ps_stenprot_146schuz_s146012.htm
Kdy: 2025-06-25
Co: e Řebíčka za ODS a Víta Bárty za Věci veřejné. Ano, Aleš Řebíček, to byl ten, co měl stavební firmu a byl ministr dopravy. Že je skvělá ODS? Berger zasáhl znovu a konstatoval ve znaleckém posudku, že diskety ze závadným obsahem nalezené u Hály mohl nastražit kdokoli a nelze prokázat, že data na disky nahrál konkrétně Hála. Takovou tezi by šlo uplatnit u jakéhokoliv digitálního nosiče, kdekoliv, tato ver
***
Kdo: Poslanec Patrik Nacher
Kde: 2021ps_stenprot_113schuz_s113095.htm
Kdy: 2024-10-03
Co: u. Chodili jsme do videopůjčoven s VHS kazetami, teď se díváme na Netflix a YouTube. Pracovali jsme na sálových počítačích s děrnými štítky a domácími počítači se záznamem na magnetofonové pásky a na diskety, teď máme internet, počítače s gigabytovými paměťmi a stovky megabytů v chytrých telefonech. Hráli jsme člověče, nezlob se, šachy, dámu, prší, kuličky, monopoly, později mini pasiáns na stolních poč
***

In [121]:
hledej(r"(?i)biograf",sestupne=True)

Kdo: Poslanec David Rath
Kde: 2006ps_stenprot_032schuz_s032183.htm
Kdy: 2008-06-11
Co: ým návrhem, třeba nejde k sám sobě, zamyslí se nad sebou, neřekne si vždyť já mám velké rezervy ve své práci, platí mě daňoví poplatníci, neplatí mě vůbec špatně. V době, kdy jsem provozoval soukromý biograf, tak ten mi skončil bankrotem, takže jsem rád, že mám státní dobře placené místo, tak si ho budu vážit a budu za to nějakým způsobem odvádět i určitý výkon. Jaký je výsledek? Pan poslanec Tluchoř čas
***
Kdo: Poslanec David Rath
Kde: 2006ps_stenprot_015schuz_s015048.htm
Kdy: 2007-06-06
Co:  který je takto výrazný, se ještě prohloubí. To je vaše politika. To jsou důsledky vaší politiky! Dámy a pánové, každá změna má mít nějaký smysl. Mně to připadá, že vláda je v pozici člověka, který v biografu vykřikne, že hoří, a v nastalém chaosu a zmatku odcizí několika desítkám lidem peněženky. (Tleskají poslanci sociální demokracie.) Těch několik lidí, kterým seberete ty peněženky, dokonce většině, t
***
Kd

In [125]:
hledej(r"(?i)mimozemš")

Kdo: Poslanec Josef Krejsa
Kde: 1996ps_stenprot_006schuz_s006022.htm
Kdy: 1996-11-01
Co: Paní předsedající, dámy a pánové, nedávno jsem zde byl svými dvěma kolegy napaden z nenávisti vůči jiným etnickým skupinám, jiným náboženstvím, jiným národům a málem i z nenávisti vůči mimozemšťanům, přestože (stenozáznamy mi dají zapravdu) jsem na půdě toto ctihodné sněmovny neřekl dodnes jediného slova proti cikánům, židům ani příslušníkům jiných ras a náboženství. Přesto jsem hrdý na to,
***
Kdo: Poslanec Miroslav Sládek
Kde: 1996ps_stenprot_008schuz_s008033.htm
Kdy: 1997-02-11
Co: byla to samozřejmě trochu nadsázka, jelikož nežijeme už tolik v minulosti a hledíme do budoucna. Prohlásil jsem dnes na tiskové konferenci: ano, přiznávám se k tomu, že mohu za únos agentky Skullyové mimozemšťany. Ta logika, kterou je vedena kampaň, pomlouvačná a špinící kampaň proti republikánské straně je stejná, jako je: velryba dýchá, babička dýchá, babička bude nejspíš kytovec. Nemohu zde opakovat vý
***
Kdo: Pos

In [137]:
hledej(r"(?i)formul\w{0,5} 1[^\w]")

Kdo: Poslanec Miloslav Kala
Kde: 2002ps_stenprot_030schuz_s030116.htm
Kdy: 2004-03-26
Co: a jízdy netelefonuje, a když musí v průběhu závodu odstavit vozidlo, dělá všechno pro to, aby zaparkoval mimo možnost kontaktu s ostatními účastníky závodu. Dokonce je zajímavé, že například pravidla Formule 1 upravují situaci, kdy již se změní adhezní vlastnosti vozovky deštěm a podobně, tak vyjede takzvaný safety car, zabrzdí provoz, zakáže předjíždění a třeba i ukončí závod. Tady by stálo za úvahu, proč,
***
Kdo: Ministr průmyslu a obchodu ČR Milan Urban
Kde: 2002ps_stenprot_047schuz_s047054.htm
Kdy: 2005-09-21
Co: namená to, že bychom měli být extrémní na druhou stranu, to znamená trvat na tom, když v celé Evropě bude tento proces rozhodnut, tzn. že se nepojede v Evropské unii žádný závod, ani motocyklový, ani formule 1. Na druhou stranu si myslím, že dnes ten protitabákový prapor v neprospěch motoristického sportu je přece jen trochu silné gesto. Podporoval bych tedy závěry, které v tomto smě

In [139]:
hledej(r"(?i)dálnic")

Kdo: Místopředseda Petr
Kde: 1946uns_stenprot_083schuz_s083008.htm
Kdy: 1947-11-25
Co: Žádám p. posl. Horu, aby přednesl svůj dotaz na pana ministra techniky ve věci stavby dálnice Praha - Humpolec a pražského okruhu.
***
Kdo: Posl. Hora (čte)
Kde: 1946uns_stenprot_083schuz_s083008.htm
Kdy: 1947-11-25
Co: Pane předsedo, slavná sněmovno, paní a pánové! Činím stručný dotaz podle jedn. řádu na pana ministra techniky ve věci stavby dálnice Praha - Humpolec a pražského okruhu. Poloha Československé republiky může pro nás znamenat mnohé výhody soustředění komunikačních spojů a jich vedení přes naše uzemí. Od roku 1939 bylo plánováno a z
***
Kdo: K dotazu pana posl. Hory dovoluji si odpověděti takto
Kde: 1946uns_stenprot_083schuz_s083008.htm
Kdy: 1947-11-25
Co: Stav, v jakém je stavba dálnice Praha - Humpolec, je mi dobře znám. Do konce roku 1946 bylo prostavěno 358 mil. Kčs. Ještě před rozhodnutím o dvouletém plánu vypracovalo ministerstvo techniky v oboru silničních staveb svůj pětilet


In [143]:
hledej(r"(?i)vysokorychlostn\w{0,10} (želez|vla)")

Kdo: Místopředseda vlády a ministr dopravy ČR Milan Šimonovský
Kde: 2002ps_stenprot_048schuz_s048154.htm
Kdy: 2005-10-18
Co: o úřadu o ukládání tzv. ochranných opatření v rámci ochrany trhu před nebezpečnými výrobky v oblasti součástí drah, kterou se dokončí také transpozice směrnice Rady o interoperabilitě transevropského vysokorychlostního železželez
***
Kdo: Poslanec Martin Kolovratník
Kde: 2013ps_stenprot_023schuz_s023040.htm
Kdy: 2014-12-03
Co:  s vicepremiérem a ministrem financí řešili intenzivně dálnici D8 a onen zával. Co se týká železnice, máme ambiciózní plány a chystáme ve velmi krátké době rozjet přípravu vysokorychlostních spojení, vysokorychlostních železželez
***
Kdo: Poslanec Ivan Pilný
Kde: 2013ps_stenprot_023schuz_s023212.htm
Kdy: 2014-12-10
Co:  výše bude známá, až se uzavře rok 2014. Přibližně se jedná o něco mezi dvěma až třemi miliardami, které by tam mohly přijít, možná i víc. To je jedna poznámka. Druhá, která se týká peněz na projekty vysokorychlostních železž

In [145]:
hledej(r"(?i)rychlovla")

Kdo: Posl. Biňovec
Kde: 1918ns_ps_stenprot_072schuz_s072004.htm
Kdy: 1919-09-16
Co: ou dopraviti do Prahy. To si musíme říci: Vilémovi dovedli sloužit, ale naší republice sloužit nedovedou. (Hluk.) Tak to vypadá! Podívejte se do těch stanic a budete vidět! Jedete osobním vlakem nebo rychlovlakem, ale nepojedete pořádně z jedné stanice do druhé. To nejsou vinni železniční zřízenci! Úředníci ve své pasivní resistenci proti republice provádějí ty věci. Řekněte to, co nám říkáte, do svých vla
***
Kdo: Ministr železnic dr. Franke
Kde: 1918ns_ps_stenprot_096schuz_s096002.htm
Kdy: 1919-12-10
Co: žen do něho tendr. Následkem této srážky byla přerušena veškerá doprava na této trati až do 12 hodin v noci, kdy vlakem č. 420 přestupováním na místě nehody byla doprava osobních vlaků zase zavedena. Rychlovlaky vedeny byly přes Vysočany-Všetaty-Lysou. Aby mohly býti co nejdříve vypraveny pomocné vlaky k ošetření raněných, bylo použito souprav vlaku č. 136 a 137, jež následkem nehody ve Mstěticích a v

In [147]:
hledej(r"VRT")

Kdo: Místopředseda PSP Tomio Okamura
Kde: 2017ps_stenprot_007schuz_s007086.htm
Kdy: 2018-03-01
Co: dpověděl na interpelaci pana poslance Zbyňka Stanjury ve věci aktuálního stavu přípravy a aktuálního stavu vyhlášených výběrových řízení na zpracování studií proveditelnosti na vysokorychlostní tratě VRT v České republice. Interpelace se spolu s odpovědí předkládá jako sněmovní tisk číslo 110. Otevírám rozpravu a slovo má opět pan poslanec Zbyněk Stanjura. Pane poslanče, máte slovo.
***
Kdo: Předseda PSP Radek Vondráček
Kde: 2017ps_stenprot_012schuz_s012080.htm
Kdy: 2018-04-12
Co: avy odpověděl na interpelaci poslance Zbyňka Stanjury ve věci aktuálního stavu přípravy a aktuálního stavu vyhlášených výběrových řízení na zpracování studií proveditelnosti na vysokorychlostní tratě VRT v České republice. Interpelace se spolu s odpovědí předkládá jako sněmovní tisk číslo 101. Poslanecká sněmovna 1. března 2018 na návrh poslance Zbyňka Stanjury přerušila projednávání této odpovědi do
***
Kdo: Mí

In [155]:
hledej(r"(?i)vzducholo",sestupne=True)

Kdo: Poslanec Richard Brabec
Kde: 2021ps_stenprot_045schuz_s045114.htm
Kdy: 2022-11-30
Co:  Sněmovny, tak speciálně vy, pane ministře, protože si vás všichni pamatujeme jako velmi zdatného rétora, který jste neopomenul jakoukoliv mezírku využít, byste visel u stropu této krásné budovy jako vzducholoď Zeppelin a vyčítal byste nám to - a právem, protože si neumím představit (Potlesk poslanců Hnutí ANO.), že bych přišel jako ministr, který by měl znát každou korunu rozpočtu své kapitoly, a někde js
***
Kdo: Poslanec Michal Doktor
Kde: 2006ps_stenprot_048schuz_s048378.htm
Kdy: 2009-02-20
Co: htěl bych jen zdůraznit, že to, co já absolutně podtrhuji a vítám v návrhu vlády, je fakt, že vláda si nebude vybírat ve svých opatřeních a ve svých návrzích, jestli má raději výrobce aut, letadel či vzducholodí. Já myslím, že ty veškeré akce vlády, ať jakékoliv, mají být celoplošné, mají stabilizovat ekonomiku jako celek, a v tomto ohledu zdůrazňuji, že podporuji a podporoval jsem vznik konceptu úlev 

In [159]:
hledej(r"(?i)[^\w]porno")

Kdo: Posl. prof. Horáček
Kde: 1918ns_ps_stenprot_094schuz_s094007.htm
Kdy: 1919-12-04
Co: , já nejsem znalec, to se ale teprve ukáže. Bylo také proti mému návrhu namítáno z odborných kruhů, že tím otevřeme možnost jisté nekalé soutěži, že pak bude se prodávati nějaký literární brak, nějaké pornografické spisy a nevím jaké. Pánové, to je všecko možné, ačkoliv nevím, jestli dnešní koncesní system tomu zcela zabraňoval. Ale jest jiný obor zákonodárství, který má právě těm vadám čeliti. To jest 
***
Kdo: Posl. Němeček
Kde: 1918ns_ps_stenprot_094schuz_s094008.htm
Kdy: 1919-12-04
Co: oráček poukázal k tomu, že dobrý tiskový zákon bude mít úlohu, vyvarovat nejenom nebezpečí, na které bylo upozorňováno se strany interesentů knihkupeckých a knihtiskařských, totiž, aby nebyla vydávána pornografie a aby uvolněním těchto živností
***
Kdo: Posl. dr. Buday
Kde: 1920ns_ps_stenprot_134schuz_s134002.htm
Kdy: 1922-04-04
Co: y, ale za zákonom eště chránené katolícke školy cenzor nepovolí. Slávna snemovno

In [161]:
hledej(r"(?i)počítač")

Kdo: Posl. inž. Smékal
Kde: 1960ns_stenprot_020schuz_s020003.htm
Kdy: 1963-07-09
Co:  řízení výroby je nesporná. Vyšla řada pojednání, kde všude lze použít počítací stroje a přesto máme značné zpoždění v jejich používání, jak je zřejmé z toho, že např. ve Spojených státech připadá 50 počítačů na 1 mil. obyvatel, ve Švýcarsku 30 
***
Kdo: Posl. inž. Smékal
Kde: 1960ns_stenprot_023schuz_s023015.htm
Kdy: 1964-01-30
Co: ozvoje a výroby výpočetní techniky. Tento úkol vytvoří především předpoklady pro zvýšení úrovně vědeckotechnických prací i pro řízení národního hospodářství. Avšak vzhledem k tomu, že vývoj středních počítačů Epos I. a Epos II. se zpozdil, nebude splněno usnesení vlády z r. 1961 pokud jsou termíny upraveny podle skutečných možností v plnění úkolů." Tolik ve zprávě. Podle mého názoru ve zprávě mělo být sp
***
Kdo: Předseda vlády Lenárt
Kde: 1964ns_stenprot_001schuz_s001011.htm
Kdy: 1964-06-23
Co: ích rokov budeme rýchlejšie mechanizovať a automatizovať oblasť riadenia a sprá

In [163]:
hledej(r"(?i)kybernet")

Kdo: Zpravodajca posl. Valo
Kde: 1954ns_stenprot_036schuz_s036006.htm
Kdy: 1960-02-17
Co: díciu a skúsenosti. To je podklad, na základe ktorého je možno dosiahnuť divy. Rozvoj chémie ide vo svete míľovými krokmi vpred a myslím, že sa oprávnene hovorí, že naše storočie je storočím chémie a kybernetiky. I náš rozpočet ukazuje, aký dôležitý význam kladieme na rozvoj chemického priemyslu, kde sme práve v posledné roky zaostávali. Rozvoj strojárenstva je úzko spätý s rozvojom plastických hmôt a ché
***
Kdo: Posl. Homola
Kde: 1960ns_stenprot_004schuz_s004018.htm
Kdy: 1960-11-17
Co: a mohla být uplatněna zásada "každý podle svých schopností, každému podle jeho potřeb", je nutné vytvořit vysoce výkonné průmyslové a zemědělské hospodářství s automatizací, chemizací, elektrifikací, kybernetizací, s vysokou mechanizací, je zapotřebí stále nových objevů vědy i velkých činů v literatuře, umění, v citovém a společenském životě člověka. Komunistická společnost, ke které spějeme, potřebuje ne
***
Kdo:

In [167]:
hledej(r"(?i)čest práci",sestupne=True)

Kdo: Poslanec Michael Rataj
Kde: 2021ps_stenprot_139schuz_s139065.htm
Kdy: 2025-05-30
Co: potřebu vyváženosti. Orgány činné v trestním řízení musí odlišovat skutečné extremistické projevy od neškodných, nostalgických či ironických odkazů na minulost; například zpěv internacionály, pozdrav čest práci či nošení komunistických řádů a odznaků a podobně. Nevhodná kriminalizace by mohla paradoxně přispět k opětovné popularizaci této ideologie, a to nikdo z nás nechce. Věřím, že podobně jako v případě 
***
Kdo: Poslankyně Kristýna Zelienková
Kde: 2013ps_stenprot_031schuz_s031175.htm
Kdy: 2015-09-17
Co:  logické. Takže svým komunistickým kolegům doporučuji, aby přestali trestat Ukrajince za to, že jejich komunističtí politici porušují zákon, a aby s námi dnes asociační smlouvu jednomyslně schválili. Čest práci a pojďme, prosím vás, hlasovat. Děkuji. (Potlesk napříč sálem, ne zleva.)
***
Kdo: Poslanec Miroslav Sládek
Kde: 1996ps_stenprot_008schuz_s008067.htm
Kdy: 1997-02-13
Co: ána v dobrém slo

In [171]:
hledej(r"(?i)poslední bitva vzp",sestupne=True)

In [173]:
hledej(r"(?i)své okovy",sestupne=True)

Kdo: Ministr kultury ČR Pavel Dostál
Kde: 1998ps_stenprot_031schuz_s031031.htm
Kdy: 2001-01-12
Co: Děkuji. Pane předsedající, já nebudu reagovat na slova týkající se mé rezignace. To je věcí Poslanecké sněmovny nebo pana premiéra, samozřejmě. Navíc já bych mohl jen ztratit své okovy. Chtěl bych jenom reagovat na slova pana poslance Filipa v tom smyslu, že bych chtěl bránit vládu, neboť vláda nepřijala nic v rozporu s ústavou nebo Listinou základních práv a svobod. Pan předseda F
***
Kdo: Posl. Horn
Kde: 1948ns_stenprot_057schuz_s057004.htm
Kdy: 1951-07-12
Co: . Systém kapitalistické výroby právě s těmito masami vykořisťovaných tvořil sám o sobě přirozenou organisací dělnické třídy, které stačilo dát třídní vědomí toho, že proletariát nemá co ztratit, leda své okovy, získat však může všechno, třídní vědomí společenského poslání proletariátu, aby se stal pro buržoasii nebezpečnou sopkou. Proto čím více proletariát rostl, tím více vyvíjela buržoasie úsilí, aby zab
***
Kdo: A není možno vy

In [177]:
hledej(r"(?i)keťas",sestupne=True)

Kdo: Poslanec Jiří Kobza
Kde: 2021ps_stenprot_108schuz_s108039.htm
Kdy: 2024-06-25
Co:  naší planetu před tím, aby zamrzla. Návrh jsem podal, to ostatní dávám k zamyšlení, protože snižovat počet povolenek na trhu, aby se zvýšila jejich cena, tomuto obchodnímu způsobu se po válce říkalo keťasení a není to nic jiného než prachsprostá šmelina. Děkuji vám.
***
Kdo: Předseda vlády ČR Mirek Topolánek
Kde: 2006ps_stenprot_027schuz_s027089.htm
Kdy: 2008-01-31
Co: u potravin, kde vůbec být nemusely, tak platí, že u televizních poplatků a nájmu jde o dílo vaší legislatury nebo vaší vlády, minulé Sněmovny. Konkrétně u potravin musí být zcela evidentní, že po tom keťasovském nárůstu na konci roku ceny jdou dolů, vracejí se na svoji řekl bych tržní úroveň v tržně pokrouceném prostředí, a je zcela evidentní, že vláda na to neměla absolutně žádný vliv, že cenové vlivy
***
Kdo: Posl. Malík
Kde: 1954ns_stenprot_010schuz_s010014.htm
Kdy: 1956-02-09
Co: m svedčia desiatky a stovky žiadostí o prídel v Mot